In [ ]:
# from langchain_community.document_loaders import UnstructuredURLLoader
# urls = ['https://www.victoriaonmove.com.au/local-removalists.html','https://victoriaonmove.com.au/index.html','https://victoriaonmove.com.au/contact.html']
# loader = UnstructuredURLLoader(urls=urls)
# data = loader.load()  

In [1]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:/Users/tjwin/Downloads/Tips for Users of Assistive Technology.pdf")

data = loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000)
docs = text_splitter.split_documents(data)

print("total number of documents: ",len(docs))

total number of documents:  15


In [3]:
docs[0]

Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2021-05-12T20:59:54-04:00', 'author': 'Long Nguyen', 'moddate': '2021-05-13T10:27:47-05:00', 'title': 'Articulate Rise Accessibility Support Documentation', 'source': 'C:/Users/tjwin/Downloads/Tips for Users of Assistive Technology.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Articulate Rise Accessibility Support Documentation  \nWhat is this document? \nOur goal is to develop the most accessible eLearning possible within the design constraints of our \ncurriculum authoring tool, Rise360. For users of Assistive Technology, we have created the following \nAccessibility Support Document to outline some of the behaviors you may observe when completing this \ntraining. \nKeyboard Navigation Resource \n• Learners can navigate Rise 360 courses entirely with their keyboards. The Rise360 Keyboard \nNavigation Resource article explains how.

In [4]:
from key import gooogle_api_key
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

c:\Users\tjwin\anaconda3\envs\gem_learn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=gooogle_api_key,timeout = 120 )

In [6]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# import numpy as np

# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-gecko-001",google_api_key=gooogle_api_key) # Or your model

# all_texts = data # Your list of documents/texts
# batch_size = 10  # Experiment with different batch sizes

# all_embeddings = []
# for i in range(0, len(all_texts), batch_size):
#     batch = all_texts[i:i + batch_size]
#     try:
#         batch_embeddings = embeddings.embed_documents(batch)
#         all_embeddings.extend(batch_embeddings)
#     except Exception as e:
#         print(f"Error embedding batch {i}-{i+batch_size}: {e}")
#         # Handle the error, e.g., retry, skip, or save the failed batch
#         # Example: time.sleep(5) # Wait and retry
#         # Example: continue # Skip this batch

# all_embeddings = np.array(all_embeddings) # Convert to numpy array if needed

In [7]:
vectorstore = Chroma.from_documents(documents=docs,embedding=embedding_model)

In [8]:
retriver = vectorstore.as_retriever(search_type = "similarity",search_kwags = {"k":6})

k= retriver.invoke("what is the title of document")

In [9]:
print(k[1].page_content)

• The interactive markers might not have a meaningful name. The alternative text of the 
background image before the interactive markers will provide learners with context of the 
interaction. 
• For some screen reader learners, the container for the marker controls can receive keyboard 
focus. 
• The interactive markers have repeated circle animations around them. The animations cannot 
be paused, stopped, or hidden. 
• Some Dragon users might not be able to close the interactive markers’ dialogs using direct 
command. Dragon users need to say “Click plus” commands to close the dialogs. 
Accordion (Expand/Collapse) 
• The accordion component can be expanded/collapsed to reveal additional information about a 
topic.  
• For some screen reader learners, the accordion expand/collapse state is not communicated 
immediately. Learners would need to use the up and down arrow keys to check the state. 
Note Block


In [10]:
llm = ChatGoogleGenerativeAI(
    google_api_key=gooogle_api_key,  # Fixed variable name
    model="gemini-1.5-pro",
    temperature=0.2,
    #max_output_tokens=500,
    max_tokens = 500
)

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



In [14]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [15]:
question_answer_chain = create_stuff_documents_chain(llm,prompt=prompt)
rag_chain = create_retrieval_chain(retriver,question_answer_chain)

In [17]:
response = rag_chain.invoke({"input": "what is the conclusion of the document"})
print(response["answer"])

This document outlines accessibility features and limitations within Rise360 eLearning courses.  It details keyboard navigation, zoom functionality, and specific behaviors of knowledge checks and interactive elements.  The goal is to provide assistive technology users with information to navigate the courses effectively.
